In [ ]:
%matplotlib inline

import matplotlib.pyplot as plt

import numpy as np
np.set_printoptions(suppress=True) 

from aif360.datasets import BinaryLabelDataset
from aif360.datasets import AdultDataset, GermanDataset, CompasDataset
from aif360.metrics import BinaryLabelDatasetMetric, ClassificationMetric
from aif360.metrics.utils import compute_boolean_conditioning_vector
from aif360.algorithms.preprocessing.optim_preproc_helpers.data_preproc_functions\
            import load_preproc_data_adult
from common_utils import compute_metrics

from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler

## utility functions
from common_utils import compute_metrics ## taken from AIF360 github repo

## for Reweighting
from aif360.algorithms.preprocessing.reweighing import Reweighing 

## Optimized Preprocessing
from aif360.algorithms.preprocessing.optim_preproc import OptimPreproc
from aif360.algorithms.preprocessing.optim_preproc_helpers.distortion_functions\
            import get_distortion_adult
from aif360.algorithms.preprocessing.optim_preproc_helpers.opt_tools import OptTools

## Learning Fair Representations
from aif360.algorithms.preprocessing.lfr import LFR

## Load the data

In [ ]:
## p 113
priv_group   = [{'sex': 1}]
unpriv_group = [{'sex': 0}]
census_data  = load_preproc_data_adult(['sex']) ## utility function to collapse categories
                                                ## according to details of dataset

## Utility functions: splitting data, building models

In [ ]:
## p 113
def split_data_trn_vld_tst(data_raw):
    dset_raw_trn, dset_raw_vt  = data_raw.split(   [0.7], shuffle = True)
    dset_raw_vld, dset_raw_tst = dset_raw_vt.split([0.5], shuffle = True)
    
    return dset_raw_trn, dset_raw_vld, dset_raw_tst

In [ ]:
## p 121
def build_logit_model(dset_trn, dset_tst, privileged_groups, unprivileged_groups):
    
    scaler = StandardScaler()
    X_trn  = scaler.fit_transform(dset_trn.features)
    y_trn  = dset_trn.labels.ravel()
    w_trn  = dset_trn.instance_weights.ravel()
    
    lmod = LogisticRegression()
    lmod.fit(X_trn, y_trn, 
             sample_weight = w_trn)

    dset_tst_pred        = dset_tst.copy(deepcopy=True)
    X_tst                = scaler.transform(dset_tst_pred.features)
    dset_tst_pred.labels = lmod.predict(X_tst)
    
    print("HOMEMADE METRICS")
    priv_idx   = np.where(dset_tst_pred.protected_attributes.ravel() == 1.0)[0]
    unpriv_idx = np.where(dset_tst_pred.protected_attributes.ravel() == 0.0)[0]
    
    print(np.sum(dset_tst_pred.labels[priv_idx] == 1.0) / 
          np.sum(dset_tst_pred.labels[priv_idx] > -1.0))
    print(np.sum(dset_tst_pred.labels[unpriv_idx] == 1.0) / 
          np.sum(dset_tst_pred.labels[unpriv_idx] > -1.0))
    print("Mean difference: %0.2f" % 
          (np.mean(dset_tst_pred.labels[unpriv_idx]) - np.mean(dset_tst_pred.labels[priv_idx])))
    print("Disparate impact: %0.2f" % 
          (np.mean(dset_tst_pred.labels[unpriv_idx]) / np.mean(dset_tst_pred.labels[priv_idx])))

    
    metric_tst = BinaryLabelDatasetMetric(dset_tst_pred, unprivileged_groups, privileged_groups)
    print("PREROLLED METRICS")
    print(metric_tst.num_positives(privileged = True) / metric_tst.num_instances(privileged = True))
    print(metric_tst.num_positives(privileged = False) / metric_tst.num_instances(privileged = False))
    print("Disparate impact is %0.2f (closer to 1 is better)" % metric_tst.disparate_impact())
    print("Mean difference  is %0.2f (closer to 0 is better)" % metric_tst.mean_difference())

    return lmod, dset_tst_pred, metric_tst 

## Examine raw dataset and a logistic regression

In [ ]:
## p 113
# reproducibility
np.random.seed(316)

# split into train, validate, test
dset_raw_trn, dset_raw_vld, dset_raw_tst = split_data_trn_vld_tst(census_data)

In [ ]:
## p 113
## calculate the metric of interest
metric_raw_trn = BinaryLabelDatasetMetric(dset_raw_trn, 
                                         unprivileged_groups = unpriv_group,
                                         privileged_groups   = priv_group)

print("Disparate impact is   %0.2f (closed to 1 is better)" % metric_raw_trn.disparate_impact())
print("Mean difference is   %0.2f (closer to 0 is better)" % metric_raw_trn.mean_difference())

### taking a look at coefficient values

In [ ]:
dset_raw_trn.feature_names

In [ ]:
# reproducibility
np.random.seed(316)

## raw training data
raw_lmod, raw_pred, raw_metric = build_logit_model(dset_raw_trn, dset_raw_tst, priv_group, unpriv_group)

In [ ]:
## plot coefficients for a quick visual summary of values
print(dset_raw_trn.feature_names[:9])
plt.plot(raw_lmod.coef_.ravel()[:9])

In [ ]:
print(dset_raw_trn.feature_names[9:])
plt.plot(raw_lmod.coef_.ravel()[9:])

## Suppression
### p 115

In [ ]:
def build_logit_model_suppression(dset_trn, 
                                  dset_tst, 
                                  privileged_groups, 
                                  unprivileged_groups):
    
    scaler = StandardScaler()
    X_trn  = scaler.fit_transform(dset_trn.features[:, 2:])
    y_trn  = dset_trn.labels.ravel()
    w_trn  = dset_trn.instance_weights.ravel()
    
    lmod = LogisticRegression()
    lmod.fit(X_trn, y_trn, 
             sample_weight = w_trn)

    dset_tst_pred        = dset_tst.copy(deepcopy=True)
    X_tst                = scaler.transform(dset_tst_pred.features[:, 2:])
    dset_tst_pred.labels = lmod.predict(X_tst)

    metric_tst = BinaryLabelDatasetMetric(dset_tst_pred,
                                          unprivileged_groups, 
                                          privileged_groups)
    print("HOMEMADE METRICS")
    priv_idx = np.where(dset_tst_pred.protected_attributes.ravel() == 1.0)[0]
    unpriv_idx = np.where(dset_tst_pred.protected_attributes.ravel() == 0.0)[0]
    print(np.sum(dset_tst_pred.labels[priv_idx] == 1.0) / np.sum(dset_tst_pred.labels[priv_idx] > -1.0))
    print(np.sum(dset_tst_pred.labels[unpriv_idx] == 1.0) / np.sum(dset_tst_pred.labels[unpriv_idx] > -1.0))
    print("Mean difference: %0.2f" % (np.mean(dset_tst_pred.labels[unpriv_idx]) - np.mean(dset_tst_pred.labels[priv_idx])))
    print("Disparate impact: %0.2f" % (np.mean(dset_tst_pred.labels[unpriv_idx]) / np.mean(dset_tst_pred.labels[priv_idx])))

    
    metric_tst = BinaryLabelDatasetMetric(dset_tst_pred,
                                    unprivileged_groups, privileged_groups)
    print("PREROLLED METRICS")
    print(metric_tst.num_positives(privileged = True) / metric_tst.num_instances(privileged = True))
    print(metric_tst.num_positives(privileged = False) / metric_tst.num_instances(privileged = False))
    print("Disparate impact is %0.2f (closer to 1 is better)" % metric_tst.disparate_impact())
    print("Mean difference  is %0.2f (closer to 0 is better)" % metric_tst.mean_difference())
    
    return lmod, dset_tst_pred, metric_tst 

In [ ]:
# reproducibility
np.random.seed(316)

sup_lmod, sup_pred, sup_metric = build_logit_model_suppression(dset_raw_trn, dset_raw_tst, priv_group, unpriv_group)

### Suppression turns out not to be that bad, in the sense that the mean difference is reduced compared to the baseline model presented above and disparate impact is closer to 1. This result is referenced on p 117 but not adequately discussed there.

# Preprocessing via reweighting
### p 117

In [ ]:
# reproducibility
np.random.seed(316)

In [ ]:
## p 120
## transform the data set
RW = Reweighing(unprivileged_groups = unpriv_group,
                privileged_groups   = priv_group)
RW.fit(dset_raw_trn)
dset_rewgt_trn = RW.transform(dset_raw_trn)

## calculate the metric of interest
metric_rewgt_trn = BinaryLabelDatasetMetric(dset_rewgt_trn, 
                                         unprivileged_groups = unpriv_group,
                                         privileged_groups   = priv_group)
print("Difference in mean outcomes = %f" %
      metric_rewgt_trn.mean_difference())
print("Disparate impact = %f" %
      metric_rewgt_trn.disparate_impact())

In [ ]:
## 4 weights resulte because there are 4 types
## privileged/unprivileged x positive/negative outcome (2 x 2 = 4)
set(dset_rewgt_trn.instance_weights)

### Now that we have reweighted the data, fit a logistic regression with the reweighted dataset

In [ ]:
# reproducibility
np.random.seed(316)

## fairness preprocessed data
rewgt_lmod, rewgt_pred, rewgt_metric = build_logit_model(dset_rewgt_trn, dset_raw_tst, priv_group, unpriv_group)

### Dpending on your fairness metric, this does slightly better than suppression as indicated by disparate impact. note that the disparate impact however would still meet the criterion of presumptive disparate impact under the EEOC's 4/5 rule.

## Quick model comparison

In [ ]:
from scipy.stats import pearsonr
pearsonr(rewgt_lmod.coef_[0], raw_lmod.coef_[0])

#### We see a difference in how the gender variable is weighted

In [ ]:
## Gender is the second coefficient and here we plot the difference
## between the coefficients in the reweighted as compared to raw/naive model
plt.plot(rewgt_lmod.coef_[0] -  raw_lmod.coef_[0])

In [ ]:
## a scatetr plot shows that the coefficients are mostly quite close
## in value in the 2 models
plt.scatter(rewgt_lmod.coef_[0],  raw_lmod.coef_[0])

In [ ]:
## importantly we also see that the reweighting doesn't distort the values of the other coefficients
## you should be asking whether this would be true if other variables were highly correlated with the
## protected group. we will study this later in the book

### Let's look at how the models treat females vs. males in the case of the model trained on the raw data and on the preprocessed data

In [ ]:
fem_idx = np.where(dset_raw_tst.features[:, 1] == 0)[0][5]

In [ ]:
fem_test_case = np.copy(dset_raw_tst.features[fem_idx:(fem_idx + 1)]) ## funny slicing to preserve 2d 

In [ ]:
fem_test_case

In [ ]:
rewgt_lmod.predict_proba(fem_test_case)

In [ ]:
raw_lmod.predict_proba(fem_test_case)

In [ ]:
fake_male_test_case = np.copy(fem_test_case)
fake_male_test_case[0, 1] = 1.0
fake_male_test_case

In [ ]:
rewgt_lmod.predict_proba(fake_male_test_case) - rewgt_lmod.predict_proba(fem_test_case)

In [ ]:
raw_lmod.predict_proba(fake_male_test_case) - raw_lmod.predict_proba(fem_test_case)

In [ ]:
raw_lmod.classes_

### So probability of being successful goes up by almost 10% just for being male in the raw case even with all else being equal. this looks like an unfair/illegal model (but then again unlikely)  such a model would get deployed in a legally regulated area

# Learning fair representations
### p 123

In [ ]:
# reproducibility
np.random.seed(316)

TR = LFR(unprivileged_groups = unpriv_group, 
         privileged_groups = priv_group)
TR = TR.fit(dset_raw_trn)

In [ ]:
dset_lfr_trn = TR.transform(dset_raw_trn, thresh = 0.5)
dset_lfr_trn = dset_raw_trn.align_datasets(dset_lfr_trn)

dset_lfr_tst = TR.transform(dset_raw_tst, thresh = 0.5)
dset_lfr_tst = dset_raw_trn.align_datasets(dset_lfr_tst)

In [ ]:
metric_op = BinaryLabelDatasetMetric(dset_lfr_trn, 
                                      unprivileged_groups = unpriv_group,
                                      privileged_groups   = priv_group)
print("Mean difference:  %0.2f" % metric_op.mean_difference())
print("Disparate impact: %0.2f" % metric_op.disparate_impact())
print("Size %d" % dset_lfr_trn.features.shape[0])

In [ ]:
metric_op_tst = BinaryLabelDatasetMetric(dset_lfr_tst, 
                                      unprivileged_groups = unpriv_group,
                                      privileged_groups   = priv_group)
print("Mean difference:  %0.2f" % metric_op_tst.mean_difference())
print("Disparate impact: %0.2f" % metric_op_tst.disparate_impact())
print("Size %d" % dset_lfr_tst.features.shape[0])

In [ ]:
# reproducibility
np.random.seed(316)

## fairness preprocessed data
lfr_lmod1, lfr_pred, lfr_metric = build_logit_model(dset_lfr_trn, dset_raw_tst, priv_group, unpriv_group)

## Tuning additional hyperparameters
#### not covered in book

In [ ]:
thresholds = [0.3, 0.4, 0.5, 0.6, 0.7]
for thresh in thresholds:
    
    # Transform training data and align features
    dset_lfr_trn = TR.transform(dset_raw_trn, threshold = thresh)

    metric_lfr_trn = BinaryLabelDatasetMetric(dset_lfr_trn, 
                                             unprivileged_groups = unpriv_group,
                                             privileged_groups   = priv_group)

    unpriv_idx = np.where(dset_lfr_trn.protected_attributes.ravel() == 0.0)[0]
    print("Pct of positive outcomes for unpriv group: %0.3f" % 
          (np.where(dset_lfr_trn.labels[unpriv_idx] == 1.0)[0].shape[0] / unpriv_idx.shape[0]))
    
    priv_idx = np.where(dset_lfr_trn.protected_attributes.ravel() == 1.0)[0]
    print("Pct of positive outcomes for priv group: %0.3f\n" % 
          (np.where(dset_lfr_trn.labels[priv_idx] == 1.0)[0].shape[0] / priv_idx.shape[0]))

##### Preprocessing does not remove the potential for in-processing or post-processing interventions. More on this in the next two chapters.

In [ ]:
#### For example, consider whether to retain the logistic regression classification threshold at 0.5
#### even after the data has been transformed. Perhaps it should be shifted to increase accuracy 
#### in recognition that some accuracy has been sacrificed in transforming the data, maybe it can be
#### recovered in optimizing the threshold as a hyperparameter.

In [ ]:
#### Also we have only compared along group fairness metrics. We should also consider incorporating measures of
#### individual fairness. However this would be a poor example to consider individual fairness
#### given that the categories are so broad (age crossed with education). The less specific the data
#### the less compelling the interest in individual fairness.

# Preprocess by learning an optimal representation
### p 127

In [ ]:
## p 129
# reproducibility
np.random.seed(316)

optim_options = {
    "distortion_fun": get_distortion_adult,
    "epsilon": 0.05,
    "clist": [0.99, 1.99, 2.99],
    "dlist": [.1, 0.05, 0]
}

OP = OptimPreproc(OptTools, optim_options)

OP = OP.fit(dset_raw_trn)

## p 131
# Transform training data and align features
dset_op_trn = OP.transform(dset_raw_trn, transform_Y=True)
dset_op_trn = dset_raw_trn.align_datasets(dset_op_trn )

In [ ]:
metric_op = BinaryLabelDatasetMetric(dset_op_trn, 
                                      unprivileged_groups = unpriv_group,
                                      privileged_groups   = priv_group)
print("Mean difference:  %0.2f" % metric_op.mean_difference())
print("Disparate impact: %0.2f" % metric_op.disparate_impact())

In [ ]:
## Transform testing data
dset_op_tst = OP.transform(dset_raw_tst, transform_Y=True)
dset_op_tst = dset_raw_trn.align_datasets(dset_op_tst)

In [ ]:
# reproducibility
np.random.seed(316)

## fairness preprocessed data
op_lmod, op_pred, op_metric = build_logit_model(dset_op_trn, dset_op_tst, priv_group, unpriv_group)

#### We can define the distortion function/metric differently e.g. based on domain knowledge

In [ ]:
## p 132
def get_distortion_adult2(vold, vnew):
    # Define local functions to adjust education and age
    def adjustEdu(v):
        if v == '>12':
            return 13
        elif v == '<6':
            return 5
        else:
            return int(v)

    def adjustAge(a):
        if a == '>=70':
            return 70.0
        else:
            return float(a)

    def adjustInc(a):
        if a == "<=50K":
            return 0
        elif a == ">50K":
            return 1
        else:
            return int(a)

    # value that will be returned for events that should not occur
    bad_val = 3.0

    # Adjust education years
    eOld = adjustEdu(vold['Education Years'])
    eNew = adjustEdu(vnew['Education Years'])

    # Education cannot be lowered or increased in more than 1 year
    #########################################################################
    if (eNew < eOld - 1) | (eNew > eOld+1): ## CHANGED THIS TO LESS STRINGENT
        return bad_val
    #########################################################################
    # adjust age
    aOld = adjustAge(vold['Age (decade)'])
    aNew = adjustAge(vnew['Age (decade)'])

    # Age cannot be increased or decreased in more than a decade
    #########################################################################
    if np.abs(aOld-aNew) > 15.0: ## CHANGED THIS TO LESS STRINGENT
        return bad_val
    #########################################################################

    # Penalty of 2 if age is decreased or increased
    if np.abs(aOld-aNew) > 0:
        return 2.0

    # Adjust income
    incOld = adjustInc(vold['Income Binary'])
    incNew = adjustInc(vnew['Income Binary'])

    # final penalty according to income
    if incOld > incNew:
        return 1.0
    else:
        return 0.0
    
# reproducibility
np.random.seed(316)

optim_options2 = {
    "distortion_fun": get_distortion_adult2,
    "epsilon": 0.05,
    "clist": [0.99, 1.99, 2.99],
    "dlist": [.1, 0.05, 0]
}

OP2 = OptimPreproc(OptTools, optim_options2)
OP2 = OP2.fit(dset_raw_trn)

# Transform training data and align features
dset_op_trn2 = OP2.transform(dset_raw_trn, transform_Y=True)
dset_op_trn2 = dset_raw_trn.align_datasets(dset_op_trn2)

metric_op2 = BinaryLabelDatasetMetric(dset_op_trn2, 
                                      unprivileged_groups = unpriv_group,
                                      privileged_groups   = priv_group)
print("Mean difference:  %0.2f" % metric_op2.mean_difference())
print("Disparate impact: %0.2f" % metric_op2.disparate_impact())

# reproducibility
np.random.seed(316)

## fairness preprocessed data
op_lmod2, op_pred2, op_metric2 = build_logit_model(dset_op_trn2, dset_raw_tst, priv_group, unpriv_group)

#### Alternately can adjust the tolerance by upping the probability limits for the distortion

In [ ]:
# reproducibility
np.random.seed(316)

optim_options3 = {
    "distortion_fun": get_distortion_adult,
    "epsilon": 0.05,
    "clist": [0.99, 1.99, 2.99],
    "dlist": [.15, 0.10, 0.05]
}

OP3 = OptimPreproc(OptTools, optim_options)

OP3 = OP.fit(dset_raw_trn)

# Transform training data and align features
dset_op_trn3 = OP3.transform(dset_raw_trn, transform_Y=True)
dset_op_trn3 = dset_raw_trn.align_datasets(dset_op_trn3)

metric_op3 = BinaryLabelDatasetMetric(dset_op_trn3, 
                                      unprivileged_groups = unpriv_group,
                                      privileged_groups   = priv_group)
print("Mean difference:  %0.2f" % metric_op3.mean_difference())
print("Disparate impact: %0.2f" % metric_op3.disparate_impact())

## Transform testing data
dset_op_tst3 = OP.transform(dset_raw_tst, transform_Y=True)
dset_op_tst3 = dset_raw_trn.align_datasets(dset_op_tst3)

# reproducibility
np.random.seed(316)

## fairness preprocessed data
op_lmod3, op_pred3, op_metric3 = build_logit_model(dset_op_trn3, dset_op_tst3, priv_group, unpriv_group)

In [ ]:
## note we don't need to transform the test data to enjoy the benefits of the transformation
op_lmod4, op_pred4, op_metric4 = build_logit_model(dset_op_trn3, dset_raw_tst, priv_group, unpriv_group)

#### Question for ourselves: When might we find larger deviations/distortions acceptable or less acceptable when considering a potential accuracy/fairness trade off (which need not necessarily exist)?

In [ ]:
## when accuracy is not of prime importance (e.g. low stakes consumer decisions)
## don't do that on health decisions!
## or when we think the data is probably noisy anyway so that we may not be adding much
## noise compared to original data collection